In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import importlib 

from VariablesClass import VariablesClass
from SupervisorClass import  SupervisorClass
from StateClass import StateClass
import colors, plot_func, funcs_geometry, funcs_physical, funcs_ML

In [ ]:
# k_stiff = np.array([2, 1])
# k_soft = np.array([1, 0.5])
# thresh = np.array([40, 20])
# theta_ss = np.array([30, 10])
# init_buckle = np.array([-1, -1])
# desired_buckle = np.array([1, -1])

# k_stiff = np.array([4.2, 2.0, 1.0, 1.0])
# k_soft = np.array([0.2, 0.5, 0.1, 0.4])
# thresh = np.array([30, 25, 15, 10])
# theta_ss = np.array([25, 15, 10, 5])
# init_buckle = np.array([-1, 1, -1, -1])
# desired_buckle = np.array([1, -1, -1, 1])

# k_stiff = np.array([[4.2, 2.0, 1.0]])
# k_soft = np.array([[0.2, 0.5, 0.1]])
# thresh = np.array([[30, 25, 15]])
# theta_ss = np.array([[5, 15, 10]])
# init_buckle = np.array([[-1, -1, 1]])
# desired_buckle = np.array([[1, 1, -1]])

k_stiff = np.array([[1.0], [1.0]])
k_soft = np.array([[1.0], [1.0]])
thresh = np.array([[40], [20]])
theta_ss = np.array([[0], [0]])
init_buckle = np.array([[1], [1]])
desired_buckle = np.array([[-1], [-1]])

supress_prints = True

# problem = 'tau'
problem = 'Fy'

L = 1

In [ ]:
iterations = 86
alpha = 0.001

In [ ]:
import VariablesClass
importlib.reload(VariablesClass)
from VariablesClass import VariablesClass

Variabs = VariablesClass(k_stiff, k_soft, thresh, theta_ss, problem, L, supress_prints)
Variabs.set_normalizations(k_stiff, k_soft, theta_ss)

In [ ]:
# from numpy.typing import NDArray

# theta_vec = np.random.uniform(-90, 90, (iterations, Variabs.hinges))

# theta_rads = np.deg2rad(theta_vec)
# p0 = np.zeros((np.shape(theta_rads)[0], np.shape(theta_rads)[1], 2))  # origin
# # p0 = np.array([0.0, 0.0])
# # (2, hinges, iterations)
# pos_vec = np.zeros((np.shape(theta_rads)[0], np.shape(theta_rads)[1], 2))
# for i, thetas in enumerate(theta_rads):
#     print('thetas ', np.rad2deg(thetas))
#     for j, theta in enumerate(thetas):
#         print('theta ', np.rad2deg((theta)))
#         if j == 0:
#             pos_prev = p0[0, 0, :]
#         else:
#             pos_prev = pos_vec[i, j-1, :]
#         thetas_sum: float = np.sum(thetas[:j+1])
#         print('sum ', np.rad2deg(thetas_sum))
#         pos_vec[i, j, :] = pos_prev + L*np.array([-np.sin(thetas_sum), np.cos(thetas_sum)])

# p1 = p0[0, :, :] + L*np.array([-np.sin(theta_rads[0, :]), np.cos(theta_rads[1, :])])
# p2 = p1 + L*np.array([-np.sin(theta_rads[0, :]+theta_rads[1, :]), np.cos(theta_rads[0, :]+theta_rads[1, :])])
# print(pos_vec[0,:,:])

# print(p1)

# plot_func.plot_arm(Supervisor.pos_vec[0,:,:], )

In [ ]:
importlib.reload(funcs_geometry)
import SupervisorClass
importlib.reload(SupervisorClass)
from SupervisorClass import SupervisorClass

Supervisor = SupervisorClass(desired_buckle, alpha)
Supervisor.init_dataset(Variabs, iterations, problem)
if problem == 'tau':
    Supervisor.desired_tau(Variabs)
elif problem == 'Fy':
    Supervisor.desired_Fy(Variabs)

In [ ]:
theta1=0.5
theta2=0.00001
tau1 = funcs_physical.tau_hinge(theta1, desired_buckle[0], theta_ss[0], k_stiff[0], k_soft[0])
tau2 = funcs_physical.tau_hinge(theta2, desired_buckle[1], theta_ss[1], k_stiff[1], k_soft[1])

funcs_physical.Fy([theta1, theta2], [tau1, tau2])

In [ ]:
importlib.reload(plot_func)
importlib.reload(funcs_physical)

for i, thetas in enumerate(Supervisor.theta_in_t):
    if i==12:
        print('thetas ', thetas)

i=2
print('theta ', Supervisor.theta_in_t[i,:])
print('desired tau ', Supervisor.desired_tau_in_t[i])
print('position ', Supervisor.pos_in_t[i,:,:])
print('desired Fy', Supervisor.desired_Fy_in_t[i])
plot_func.plot_arm(Supervisor.pos_in_t[i,:,:], Supervisor.theta_in_t[i, :], L)

i=9
print('theta ', Supervisor.theta_in_t[i,:])
print('desired tau ', Supervisor.desired_tau_in_t[i])
print('position ', Supervisor.pos_in_t[i,:,:])
print('desired Fy', Supervisor.desired_Fy_in_t[i])
plot_func.plot_arm(Supervisor.pos_in_t[i,:,:], Supervisor.theta_in_t[i, :], L)


In [ ]:
import StateClass
importlib.reload(StateClass)
from StateClass import StateClass

State = StateClass(Variabs, Supervisor, buckle=init_buckle)

## Loop

## Measure tau - single hinge

In [ ]:
importlib.reload(funcs_geometry)
importlib.reload(funcs_ML)

if problem == 'tau':
    theta_full, tau_full_init = measure_full_response(State.buckle, Variabs.theta_ss, Variabs.k_stiff, Variabs.k_soft)
    theta_full, tau_full_desired  = measure_full_response(desired_buckle, Variabs.theta_ss, Variabs.k_stiff, Variabs.k_soft)

for i in range(iterations):

    ## Measure
    if problem == 'tau':
        Supervisor.set_theta(i)
        State.calc_tau(Variabs, Supervisor.theta)
    elif problem == 'Fy':
        Supervisor.set_pos(Variabs, i)
        State.calc_Fy(Variabs, Supervisor.thetas)
    
    ## Loss
    Supervisor.calc_loss(Variabs, State)

    ## Update
    Supervisor.calc_input_update(State, Supervisor, Variabs)
    if problem == 'tau':
        State.calc_tau(Variabs, Supervisor.input_update)
    elif problem == 'Fy':
        State.calc_Fy(Variabs, Supervisor.input_update)
    ## Material evolution during Update
    State.evolve_material(Supervisor, Variabs)

    if problem == 'tau':
        theta_full, tau_full_final = measure_full_response(State.buckle, Variabs.theta_ss, Variabs.k_stiff, Variabs.k_soft)
        # plot_func.plot_response(theta_full, tau_full_init, tau_full_final, tau_full_desired, theta_range=[-90, 90],
        #                         just_init=False)

Supervisor.loss_MSE_norm_in_t = Supervisor.loss_MSE_in_t / (Variabs.k_bar * Variabs.theta_bar)

In [ ]:
Supervisor.theta_vec

In [ ]:
if problem == 'tau':
    importlib.reload(plot_func)

    plot_func.importants(State.buckle_in_t, Supervisor.desired_buckle, 
                         Supervisor.loss_MSE_norm_in_t, Supervisor.input_update_in_t)

    plot_func.plot_response(theta_full, tau_full_init, tau_full_desired, tau_full_desired, theta_range=[-90, 90], just_init=True)
    plot_func.plot_response(theta_full, tau_full_init, tau_full_desired, tau_full_desired, theta_range=[-90, 90], just_init=False)

In [ ]:
tau_full_init

## Measure Fy - double hinge

In [ ]:
if problem == 'Fy':
    L=1
    x = -0.28
    y = 1.8
    theta1, theta2 = theta_from_xy(x, y, -init_buckle[0])
    print(np.degrees(theta1), np.degrees(theta2))
    tau1 = tau_k(theta1, [1, 1], Variabs)[0]
    tau2 = tau_k(theta2, [1, 1], Variabs)[1]
    print(tau1, tau2)
    Fy1 = Fy(theta1, theta2, tau1, tau2)
    print(Fy1)

    # Reachability check
    reach_min, reach_max = 0, 2*L
    r = np.hypot(x, y)
    if r < reach_min - 1e-9 or r > reach_max + 1e-9:
        print("WARNING: Target point is outside reachable workspace. The diagram uses clipped IK (c2 clipped).")

    plot_arm(x, y, theta1, theta2)

## Fy = tau1*sin(theta1)